In [1]:
# -*- coding: utf-8 -*-
import re
import time
import random
import math
import numpy as np
import pandas as pd
import importlib
from tinderweb import TinderAutomator
from data_interface import Allowance, Backlog
from gpt3 import Gpt3
import logging
from selenium.common.exceptions import *

TLSv1.3


In [2]:
path_prefix = "/Users/philippskudlik/local_dev/tndr/automator/"

In [3]:
STATUS_CODE = {0: "NEW",
              1: "RUNNING",
              2: "DONE",
              3: "EXPIRED",
              4: "ERRONOUS"}

In [4]:
# create logger with 'spam_application'
logger = logging.getLogger('TA')
logger.setLevel(logging.INFO)
# create file handler which logs even debug messages
fh = logging.FileHandler('run.log', 'w', 'utf-8')
fh.setLevel(logging.INFO)
logger.addHandler(fh)


In [5]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

'/Users/philippskudlik/local_dev/tndr/tndrvenv/lib/python3.9/site-packages/chromedriver_autoinstaller/103/chromedriver'

In [6]:
# initialize automator
ta = TinderAutomator()

In [7]:
# initialize allowance
allowance = Allowance(path=path_prefix+"gpt_allowance.csv")
logger.info("Remaining Tokens for today: "+str(allowance.get_tokens()))

# initialize backlog
backlog = Backlog(path=path_prefix+"backlog.csv")

INFO:TA:Remaining Tokens for today: 40


In [8]:
# collect matches
try:
    tasks = ta.generate_tasklist()
except AttributeError:
    time.sleep(4)
    tasks = ta.generate_tasklist()


In [9]:
# update backlog
backlog.update_with_tasks(tasks)

0 new matches.


In [10]:
# initialize Gpt3
gpt = Gpt3()
gpt_dryrun=False
msg_dryrun=False

In [11]:
min_date = pd.Timestamp.today()-pd.Timedelta(days=21)
name_me = "Chris"

In [12]:
backlog.data[(backlog.data.Status <= 1)]

,Status,msg_count,Name,Link,Rank
5ca1e2f9f7a5df1500ffaba862d6fe0755229101000c2d1f,1,3.0,JJ,https://tinder.com/app/messages/5ca1e2f9f7a5df...,0.0
5ca1e2f9f7a5df1500ffaba861c25f1b95bbb9010083dd6e,1,8.0,Jana,https://tinder.com/app/messages/5ca1e2f9f7a5df...,1.0
5ca1e2f9f7a5df1500ffaba862dc6ece55229101000e6c30,1,4.0,Ksenia,https://tinder.com/app/messages/5ca1e2f9f7a5df...,2.0
5ca1e2f9f7a5df1500ffaba8610b01a419edfe01008b9ecb,1,0.0,Teresa,https://tinder.com/app/messages/5ca1e2f9f7a5df...,3.0
5b58c991aaed2ef83401d3f55ca1e2f9f7a5df1500ffaba8,1,0.0,Lena,https://tinder.com/app/messages/5b58c991aaed2e...,4.0
...,...,...,...,...,...
5ca1e2f9f7a5df1500ffaba861f9005f3b5fe5010003b1d0,1,1.0,Lexi,https://tinder.com/app/messages/5ca1e2f9f7a5df...,61.0
5ca1e2f9f7a5df1500ffaba861e11a2a0acc5701009fb988,1,NaN,Jay,https://tinder.com/app/messages/5ca1e2f9f7a5df...,61.0
5ca1e2f9f7a5df1500ffaba862a401018b6c5e010049440a,1,4.0,Itzel,https://tinder.com/app/messages/5ca1e2f9f7a5df...,61.0
5ca1e2f9f7a5df1500ffaba862cf3102552291010008ed4d,1,1.0,Merve,https://tinder.com/app/messages/5ca1e2f9f7a5df...,61.0


In [13]:
start = 0
limit = 10

run_report_header = ""
new_done = 0
logger.info("Remaining tokens: %d",allowance.get_tokens())
logger.info("Open tasks: %d", sum(backlog.data.Status <= 1))
for count, (id_, task) in enumerate(backlog.data[(backlog.data.Status <= 1)][start:start+limit].iterrows()):
    # todo: same loop for erronous matches plus a coounter to indicate the retry count
    logger.info("-------------------------------------------------")
    logger.info("------------------Processing Nr. %d------------------", count+1)
    if allowance.get_tokens() <= 0:
        logger.warning("Ran out of tokens...")
        break
    # open task
    ta.get(task.Link)
    name_them = task.Name
    logger.info("------------------%s------------------",name_them)
    time.sleep(6)
    
    # get basic information
    try:
        bio, match_date = ta.read_match_info()
    except Exception as e:
        #logger.info(str(e))
        logger.info("Some error while fetching match info...")
        #backlog.data.loc[id_,"Status"] = 4
        logger.info("------------------Status: Erronous.")
        continue

    ### CHECK NACH ABBRUCHBEDINGUNGEN
    if (match_date < min_date):
        backlog.data.loc[id_,"Status"] = 3
        logger.info("------------------Status: Expired.")
        continue
        
    myturn, conversation = ta.get_conversation()
    msg_count = len(conversation)
    backlog.data.loc[id_,"msg_count"] = msg_count
    if not myturn:
        logger.info("------------------Status: Running. Still no reply...")
        break

    if msg_count >= 12:
        backlog.data.loc[id_,"Status"] = 2
        logger.info("------------------Status: Done. Message count reached! Yeay!")
        run_report_header = f"Conversation with {name_them} is ready to be taken over! (Nr. {count+1})\n"
        new_done += 1
        continue

    ### INITIATE OR CONTINUE CONVERSATION
    logger.info("------------------Status: Running. Conversing...")
    # update status
    backlog.data.loc[id_,"Status"] = 1
    # build prompt
    if msg_count==0:
        prompt = gpt.build_prompt(bio, name_them, name_me, initial=True)
    else:
        prompt = gpt.build_prompt(conversation, name_them, name_me, initial=False)
    logger.info("::PROMPT::")
    logger.info(prompt)
    # get gpt response
    reply = gpt.request(prompt, stop_sequences=[name_them+":",name_me+":",name_them+" responds"], temperature=0.9, dryrun=gpt_dryrun)
    #update token allowance
    allowance.decrement()    
    logger.info("::GPT::")
    logger.info(reply)
    ta.write_message(reply, dryrun=msg_dryrun)
    time.sleep(3)

# Create run report
run_report_base = "Run from "+str(time.ctime())+f"\nProcessed {count+1} matches.\n{allowance.get_tokens()} tokes remaining for today.\nOpen conversations: {len(backlog.data[backlog.data.Status <= 1])}"
run_report = run_report_header + ("\n" * (run_report_header != "")) + run_report_base
logger.info(run_report)

#update backlog
backlog.save()

INFO:TA:Remaining tokens: 40
INFO:TA:Open tasks: 68
INFO:TA:-------------------------------------------------
INFO:TA:------------------Processing Nr. 1------------------
INFO:TA:------------------JJ------------------
INFO:TA:------------------Status: Running. Still no reply...
INFO:TA:Run from Mon Aug  1 22:39:45 2022
Processed 1 matches.
40 tokes remaining for today.
Open conversations: 68


In [14]:
with open("run.log", "r") as f:
    content = "".join(f.readlines())
run_report = run_report+"\n\n\n"+"".join(content)

In [15]:
import yagmail
yag = yagmail.SMTP("chrssku@gmail.com", oauth2_file=path_prefix+"client_secret_701724446788-id894tinkh6b1j2q86fkkv00d3t5h8b3.apps.googleusercontent.com.json")
TO = "pskudlik@gmail.com"
yag.send(TO, "Run Report",run_report, attachments=['run.log'])
print(run_report)

Run from Mon Aug  1 22:39:45 2022
Processed 1 matches.
40 tokes remaining for today.
Open conversations: 68


Remaining Tokens for today: 40
Remaining tokens: 40
Open tasks: 68
-------------------------------------------------
------------------Processing Nr. 1------------------
------------------JJ------------------
------------------Status: Running. Still no reply...
Run from Mon Aug  1 22:39:45 2022
Processed 1 matches.
40 tokes remaining for today.
Open conversations: 68



In [16]:
ta.end_session()